# Exercícios Pyspark & SparkSQL

* Configuração

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, lit, rand
from pyspark.sql.types import StringType
import random
from pyspark.sql.functions import col

spark = SparkSession.builder.master("local[*]").appName("Exercicio Intro").getOrCreate()

24/06/06 14:01:03 WARN Utils: Your hostname, daniel-Lenovo-ideapad-330-15IKB resolves to a loopback address: 127.0.1.1; using 10.0.0.108 instead (on interface wlp2s0)
24/06/06 14:01:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/06 14:01:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/06 14:01:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


* Ler .txt e listar os 5 primeiros nomes

In [5]:
df_nomes = spark.read.csv('nomes_aleatorios.txt')
df_nomes.show(5)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `genero` cannot be resolved. Did you mean one of the following? [`_c0`].;
'Filter 'genero IN (Drama,Romance,Drama,Romance)
+- Relation [_c0#56] csv


* Alterar nome da coluna do DataFrame para nomes e apresentar as 10 primeiras linhas.

In [51]:
df_nomes.printSchema()
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")
df_nomes.show(10)

root
 |-- _c0: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



* Atribuindo ao DataFrame, a coluna Escolaridade, a coluna Pais e a coluna AnoNascimento, todas geradas com valores aleatórios, a partir de determinado intervalo.

In [52]:
niveis_escolaridade = ['Fundamental', 'Médio', 'Superior']
paises_america_do_sul = ["Argentina","Bolívia","Brasil","Chile","Colômbia","Equador","Guiana","Paraguai","Peru","Suriname","Uruguai","Venezuela","Guiana Francesa"]


def sortear_string(array):
    return random.choice(array)


sortear_string_udf = udf(sortear_string, StringType())

df_nomes = df_nomes.withColumn("Escolaridade", sortear_string_udf(lit(niveis_escolaridade)))
df_nomes = df_nomes.withColumn("Pais", sortear_string_udf(lit(paises_america_do_sul)))
df_nomes = df_nomes.withColumn("AnoNascimento", (1945 + (rand() * (2010 - 1945)).cast("int")))


* Trazer linhas do DataFrame com pessoas do século 2000 utilizando pyspark

In [53]:
df_select = df_nomes.select("*").where(col("AnoNascimento") >= 2000)
df_select.show(10)

+----------------+------------+---------------+-------------+
|           Nomes|Escolaridade|           Pais|AnoNascimento|
+----------------+------------+---------------+-------------+
|     Joy Bennett|       Médio|         Guiana|         2007|
| Howard Lazarine| Fundamental|         Guiana|         2003|
|    Leroy Strahl|       Médio|        Bolívia|         2008|
|    David Medina|    Superior|Guiana Francesa|         2007|
|     Albert Leef|    Superior|        Bolívia|         2000|
|      Daryl Page| Fundamental|           Peru|         2008|
|      Anita Ross| Fundamental|        Bolívia|         2007|
|     Sandra Todd| Fundamental|        Bolívia|         2005|
|    Juliet Liles|       Médio|        Equador|         2000|
|Cristina Sheston| Fundamental|        Uruguai|         2002|
+----------------+------------+---------------+-------------+
only showing top 10 rows



* Trazer linhas da tabela com pessoas do século 2000 utilizando spark sql

In [54]:
df_nomes.createOrReplaceTempView("pessoas")
spark.sql("select * from pessoas where AnoNascimento >= 2000").show(10)

+----------------+------------+---------------+-------------+
|           Nomes|Escolaridade|           Pais|AnoNascimento|
+----------------+------------+---------------+-------------+
|     Joy Bennett| Fundamental|         Brasil|         2007|
| Howard Lazarine|    Superior|         Brasil|         2003|
|    Leroy Strahl|    Superior|Guiana Francesa|         2008|
|    David Medina|    Superior|       Paraguai|         2007|
|     Albert Leef|       Médio|         Guiana|         2000|
|      Daryl Page|       Médio|       Paraguai|         2008|
|      Anita Ross|       Médio|Guiana Francesa|         2007|
|     Sandra Todd|       Médio|        Equador|         2005|
|    Juliet Liles|       Médio|       Paraguai|         2000|
|Cristina Sheston| Fundamental|           Peru|         2002|
+----------------+------------+---------------+-------------+
only showing top 10 rows



* Contar a quantidade de pessoas da geração Millennials existente utilizando as funções do pyspark

In [58]:
geracao_millennials = df_nomes.select(col("*")).where((col("AnoNascimento") >= 1980) & (col("AnoNascimento") <= 1994)).count()

print(f"Quantidade de pessoas da geração Millennials: {geracao_millennials}")

Quantidade de pessoas da geração Millennials: 2308171


* Contar a quantidade de pessoas da geração Millennials existente utilizando as funções do spark sql

In [59]:
spark.sql("select count(*) as geracao_millennials from pessoas where AnoNascimento >= 1980 and AnoNascimento <= 1994").show()

+-------------------+
|geracao_millennials|
+-------------------+
|            2308171|
+-------------------+



* Apresentar quantidade de pessoas de cada geração agrupadas por país, em seguida ordernar a saia por país, geração e quantidade, todos em orderm crescente.

In [57]:
df_geracoes = spark.sql("SELECT pais, CASE WHEN AnoNascimento >= 1944 AND AnoNascimento <= 1964 THEN 'Baby Boomers'WHEN AnoNascimento >= 1965 AND AnoNascimento <= 1979 THEN 'Geração X' WHEN AnoNascimento >= 1980 AND AnoNascimento <= 1994 THEN 'Millennials' WHEN AnoNascimento >= 1995 AND AnoNascimento <= 2015 THEN 'Geração Z' END AS geracao, count(*) as quantidade FROM  Pessoas group by pais, geracao")
df_geracoes.orderBy(col("pais"), col("geracao"), col("quantidade")).show(truncate=False)

+---------+------------+----------+
|pais     |geracao     |quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|236904    |
|Argentina|Geração X   |177924    |
|Argentina|Geração Z   |177962    |
|Argentina|Millennials |177522    |
|Bolívia  |Baby Boomers|236039    |
|Bolívia  |Geração X   |177689    |
|Bolívia  |Geração Z   |177197    |
|Bolívia  |Millennials |178152    |
|Brasil   |Baby Boomers|236937    |
|Brasil   |Geração X   |177663    |
|Brasil   |Geração Z   |177798    |
|Brasil   |Millennials |177674    |
|Chile    |Baby Boomers|237163    |
|Chile    |Geração X   |178325    |
|Chile    |Geração Z   |176954    |
|Chile    |Millennials |177512    |
|Colômbia |Baby Boomers|236928    |
|Colômbia |Geração X   |177964    |
|Colômbia |Geração Z   |177552    |
|Colômbia |Millennials |176903    |
+---------+------------+----------+
only showing top 20 rows

